In [ ]:
import pandas as pd     
import numpy as np 
import glob as glob

Base=[]
csv_files = glob.glob('Data_EAM&IPC/*.csv')

for filename in csv_files:
    if filename == "Data_EAM&IPC\\EAM_2020.csv":
        print("ok")
        data = pd.read_csv(filename,sep=",")   
    else:
        data = pd.read_csv(filename,sep=";")
      
    Base.append(data)


In [ ]:
EAM08,EAM09,EAM10,EAM11,EAM12,EAM13,EAM14,EAM15,EAM16,EAM17,EAM18,EAM19,EAM20 = Base
#Assign each dataframe a variable of the "Base"
Data_Merge = [EAM08,EAM09,EAM10,EAM11,EAM12,EAM13,EAM14,EAM15,EAM16,EAM17,EAM18,EAM19,EAM20]
#Create a list of the dataframe
EAM08_G,EAM09_G,EAM10_G,EAM11_G,EAM12_G,EAM13_G,EAM14_G, \
            EAM15_G,EAM16_G,EAM17_G,EAM18_G,EAM19_G,EAM20_G=\
                0,0,0,0,0,0,0,0,0,0,0,0,0
Data_Gruop = [EAM08_G,EAM09_G,EAM10_G,EAM11_G,EAM12_G,EAM13_G,EAM14_G, \
            EAM15_G,EAM16_G,EAM17_G,EAM18_G,EAM19_G,EAM20_G]

In [ ]:
#PARA CAMBIAR UN LABEL EN UNA UBICACIÓN ESPECIFICA NO ESTANDARIZADO
def change_name(Data_Merge, Number, change):
    for df in Data_Merge:
        if len(df.columns) > Number: #Verificar si el dataframe tiene al menos 4 columnas
            df.columns.values[Number] = change #Me cambia el name de el label 
    return Data_Merge

In [ ]:
#Para estandairzar los labels
def change_label(Data_Merge):
    for df in Data_Merge:
        columnas = df.columns.tolist() #Crea una lista de columnas
        nuevas_columnas = [columna.upper() for columna in columnas] # coloca en mayusculas cada columna de lista
        df.columns = nuevas_columnas #Asigna el nuevo nombre
    return Data_Merge

In [ ]:
#Función para saber si una columna esta presente en un dataframe
def Exis(Data_Merge, item):
    for i, df in enumerate(Data_Merge):
        if item not in df.columns:
            columna_ausente = True
            print(f"El item no está presente en el DataFrame {i}.")
        else:
            columna_ausente = False
            print(f"El item  está presente en el DataFrame {i}.")
    return columna_ausente

In [ ]:
#Función para saber si existen datos faltantes en la columna
def exis_colum(Data_Merge,colum):
    missing_col=0
    for i in Data_Merge:
        missing_col += i[colum].isnull().sum()
    return missing_col

In [ ]:
#Función para ordenar, por una columna 
def Sort(Column,Data_Merge):
    for i, d in enumerate(Data_Merge):
        sorted_df = d.sort_values(by=Column)
        Data_Merge[i] = sorted_df 
    return Data_Merge

In [ ]:
def calculate_avg_or_sum(Column, SUMATION, UBACATION, Data_Gruop, Data_Merge, operation):
    for i, df in enumerate(Data_Gruop):
        for index in df.index:
            if index in Data_Merge[i][Column].unique():
                total = Data_Merge[i][Data_Merge[i][Column] == index][SUMATION]
                if operation == 'mean':
                    total=total.mean()
                    df.loc[index, UBACATION] = total
                elif operation == 'sum':
                    total=total.sum()
                    df.loc[index, UBACATION] = total
                elif operation == 'count':
                    total=total.value_counts()           
                    df.loc[index, UBACATION] = int(total)           
                elif operation == 'coe_sim':
                    total=total.skew()  #Da como resultado el coeficiente de asimetria de fisher
                    df.loc[index, UBACATION] = total
                elif operation == "med":
                    total=total.median()
                    df.loc[index, UBACATION] = total
                elif operation =="rango":
                    total = total.max() - total.min()
                    df.loc[index, UBACATION] = total
    return Data_Gruop

In [ ]:
def sum_of_rows(row,Colum1,Colum2):
    return row[Colum1] + row[Colum2]

In [ ]:
def mul_of_rows(row,Colum1,Colum2):
    return row[Colum1] * row[Colum2]

In [ ]:
def IHH(Column, Divisor, Numerator, Ubac, Data_Gruop, Data_Merge):
  for i, df in enumerate(Data_Gruop):
    for index in df.index:
      if index in Data_Merge[i][Column].unique():               
        Numerator = Data_Merge[i].set_index(Column)[Divisor]
        Denominator = df["SUM-WORKERS"]
        Total = (Numerator / Denominator) * 100
        Data_Merge[i].loc[Data_Merge[i][Column], Ubac] = Total
  return Data_Merge

In [ ]:
Data_Merge = change_name(Data_Merge,3,"CIIU")

In [ ]:
Data_Merge = change_label(Data_Merge)

In [ ]:
exis_colum(Data_Merge,"CIIU")
exis_colum(Data_Merge,"C4R4C9T")
exis_colum(Data_Merge,"C4R4C10T")

In [ ]:
Data_Merge=Sort("CIIU",Data_Merge)

In [ ]:
#Para crear un data frame agrupado de una base de datos
for i in range (len(Data_Merge)):
    index=Data_Merge[i]["CIIU"].drop_duplicates().sort_values()
    Data_Gruop[i]= pd.DataFrame(index=index,columns=["SUM-WORKERS_M","SUM-WORKERS_F","SUM-WORKERS","MED-WAGE","IHH","Count_CIIU",'Simetria',"rango"])


In [ ]:
Data_Gruop=calculate_avg_or_sum("CIIU","C4R4C9T","SUM-WORKERS_F",Data_Gruop,Data_Merge,"sum")
Data_Gruop=calculate_avg_or_sum("CIIU","C4R4C10T","SUM-WORKERS_M",Data_Gruop,Data_Merge,"sum")
Data_Gruop=calculate_avg_or_sum("CIIU","C3R2C3","MED-WAGE",Data_Gruop,Data_Merge,"med")

In [ ]:
for df  in Data_Gruop:
    df["SUM-WORKERS"] = df.apply(lambda row: sum_of_rows(row,"SUM-WORKERS_M","SUM-WORKERS_F"), axis=1)

In [ ]:
for df  in Data_Merge:
    df["WORKERS"] = df.apply(lambda row: sum_of_rows(row,"C4R4C10T","C4R4C9T"), axis=1) #Sumar las filas
    df["IHH_ONE"] = None

In [ ]:
Data_Merge=IHH("CIIU","WORKERS","SUM-WORKERS","IHH_ONE",Data_Gruop,Data_Merge) #Aquí estoy sacando el ihh

In [ ]:
Data_Gruop=calculate_avg_or_sum("CIIU","IHH_ONE","IHH",Data_Gruop,Data_Merge,"sum") ##Sumo el IHH individual

In [ ]:
Data_Gruop=calculate_avg_or_sum("CIIU","CIIU","Count_CIIU",Data_Gruop,Data_Merge,"count")   ## Cuanto la cantidad de empresas que hay ´por CIIU 

In [ ]:
Data_Gruop=calculate_avg_or_sum("CIIU","C3R2C3","Simetria",Data_Gruop,Data_Merge,"coe_sim")  #Miro el coeficiente de simetria

In [ ]:
Data_Gruop=calculate_avg_or_sum("CIIU","C3R2C3","rango",Data_Gruop,Data_Merge,"rango")

In [ ]:
for i, df in enumerate(Data_Gruop):
    df["IHH_cuadrado"] = df["IHH"] ** 2

    conditions = [
        df["IHH_cuadrado"] <= 1000,
        (1000 <= df["IHH_cuadrado"]) & (df["IHH_cuadrado"] < 1800),
        df["IHH_cuadrado"] >= 1800
    ]
    values = [1, 2, 3]

    df["IHH_resultado"] = np.select(conditions, values, default=np.nan)

    #Hay un problema y es que son 3 .-.

In [ ]:
exis_colum(Data_Gruop,"IHH")

In [ ]:
#Promedios Ponderados  -- para calcular 
#Para el salario se saca la mediana pagada por cada sector 
# Para saber si el promedio es un buen estadistico se realiza un examen de simetria
#Datos de estilizados -- Indicador: 

Se comienza a trabajar con los datos sacados por hechos estilizados

Se imprime una grafica de barras del IHH por sector para todos los años

In [ ]:
def count_IHH(Grupo_datos, df,SUM):
    for i, data_frame in enumerate(Grupo_datos):
        conteo = data_frame[SUM].value_counts()
        for valor in conteo.index:
            conteo_valor = conteo.loc[valor]
            df.at[df.index[i], valor] = conteo_valor
    return df

In [ ]:
def create_index(Data_Gruop):
    Index=[]
    for i in range(len(Data_Gruop)):
        w = 2008 + i 
        Index.append(w)
    return Index

In [ ]:
Index = create_index(Data_Gruop)
Columns = [1, 2, 3]
df = pd.DataFrame(index=Index, columns=Columns)


In [ ]:
df=count_IHH(Data_Gruop,df,"IHH_resultado")

In [ ]:
# import matplotlib.pyplot as plt

# # Crear una figura para el gráfico
# plt.figure()

# # Graficar los resultados de IHH_resultado con un gráfico de barras
# df.plot(kind='bar', width=0.8)

# # Etiquetas y título del gráfico
# plt.xlabel("Años")
# plt.ylabel("Cantidad de Sectores")
# plt.title("Resultados de IHH por Índice")


# # Mostrar el gráfico
# plt.show()
#plt.savefig('grafico_Sectores.jpg', format='jpg')


Se imprime una grafica con la media  del salario medio pagado por sectores con poder y sin poder.

In [ ]:
Prom = create_index(Data_Gruop)
X=['Salario_Medio']  # Acá se crea la columna vacia
Promedio_years= pd.DataFrame(index=Index, columns=X)

#Creamos un dataframe donde agrupa la mediana de cada CIIU para cada año

In [ ]:
# Leer el archivo de Excel
IPC_doc = pd.read_excel("Data_EAM&IPC\\IPC.xlsx", header=0)

# Crear un DataFrame IPC a partir de los datos leídos
IPC = pd.DataFrame({'Año': IPC_doc.iloc[:, 0], 'IPC': IPC_doc.iloc[:, 1]})

Index = create_index(Data_Gruop)

# Establecer el índice del DataFrame IPC usando la lista Index
IPC.set_index('Año', inplace=True)

# Reemplazar el índice existente con la lista Index
IPC.index = Index


In [ ]:
T=[]
for i in Data_Gruop:
    X = i['MED-WAGE'].median()
    T.append(X)

In [ ]:
Promedio_years['Salario_Medio'] = T
Promedio_years["InflaciónIPC"] = IPC["IPC"]
Promedio_years['Salario_Real'] = (Promedio_years["Salario_Medio"]  / Promedio_years["InflaciónIPC"]) *100


In [ ]:
# import matplotlib.pyplot as plt

# # Supongamos que 'df' es tu DataFrame con una columna "Datos" y un índice deseado en el eje X

# # Obtén los valores del índice y los datos de las columnas
# x_values = Promedio_years.index
# y_values_1 = Promedio_years['Salario_Real']
# y_values_2 = Promedio_years['Salario_Medio']

# # Crea la figura y los ejes
# fig, ax1 = plt.subplots()

# # Crea un segundo eje Y que comparta el mismo eje X
# ax2 = ax1.twinx()

# # Trazar datos en el primer eje Y
# ax1.plot(x_values, y_values_1, marker='o', linestyle='-', color='b', label='Salario Real')
# ax1.set_xlabel('Años')
# ax1.set_ylabel('Mediana del Salario REAL', color='b')

# # Trazar datos en el segundo eje Y
# ax2.plot(x_values, y_values_2, marker='s', linestyle='--', color='r', label='Salario Nominal')
# ax2.set_ylabel('Salario Medio', color='r')

# # Agregar leyendas para las líneas
# ax1.legend(loc='upper left', bbox_to_anchor=(0.05, 0.95))
# ax2.legend(loc='upper left', bbox_to_anchor=(0.05, 0.87))

# # Título del gráfico
# plt.title('Mediana del Salario y Salario Medio Real')

# # Muestra el gráfico
# plt.show()
# #plt.savefig('grafico_Salario.jpg', format='jpg')


Agregar el Salario minimo y el porcentaje pagado del salario minimo por la EAM y de esa medida se puede establecer el potencial de ingresos que tiene el sector

Construcción de los hechos estilizados para el empleo

In [ ]:
# Leer el archivo de Excel
import pandas as pd     
import numpy as np 
import glob as glob

csv_files_gD = glob.glob('Data_GEIH\\Desocupados\\*.[Cc][Ss][Vv]', recursive=True)
Dog_D =[]

for filename in csv_files_gD:
    if "Desocupados2008.csv" in filename:
        data_gD = pd.read_csv(filename, sep=",", encoding='latin-1', error_bad_lines=False)
    if "Desocupados2017.csv" in filename:
        data_gD = pd.read_csv(filename, sep=",", encoding='latin-1', error_bad_lines=False)
    elif "Desocupados2020.CSV" in filename:
        data_gD = pd.read_csv(filename, sep=",", encoding='latin-1', error_bad_lines=False)

    else:
        data_gD = pd.read_csv(filename,sep=";",encoding='latin-1', error_bad_lines=False)
        
    Dog_D.append(data_gD)

GEIH_D08,GEIH_D09,GEIH_D10,GEIH_D11,GEIH_D12,GEIH_D13,GEIH_D14, \
    GEIH_D15,GEIH_D16,GEIH_D17,GEIH_D18,GEIH_D19,GEIH_D20 = Dog_D
GEIH_LIST_D=[GEIH_D08,GEIH_D09,GEIH_D10,GEIH_D11,GEIH_D12,GEIH_D13,GEIH_D14, \
    GEIH_D15,GEIH_D16,GEIH_D17,GEIH_D18,GEIH_D19,GEIH_D20]

In [ ]:
csv_files_g = glob.glob('Data_GEIH\\Ocupados\\*.[Cc][Ss][Vv]', recursive=True)
Dog =[]

for filename in csv_files_g:
    if "Ocupados2008.csv" in filename:
        data_g = pd.read_csv(filename, sep=",", encoding='latin-1', error_bad_lines=False)   
    elif "Ocupados2011.csv" in filename:
        data_g = pd.read_csv(filename, sep=",", encoding='latin-1', error_bad_lines=False)    
    elif "Ocupados2017.csv" in filename:
        data_g = pd.read_csv(filename, sep=",", encoding='latin-1', error_bad_lines=False)
    elif "Ocupados2020.CSV" in filename:
        data_g = pd.read_csv(filename, sep=",", encoding='latin-1', error_bad_lines=False)
    else:
        data_g = pd.read_csv(filename,sep=";",encoding='latin-1', error_bad_lines=False)
        
    Dog.append(data_g)

GEIH_O08,GEIH_O09,GEIH_O10,GEIH_O11,GEIH_O12,GEIH_O13,GEIH_O14,GEIH_O15,GEIH_O16,GEIH_O17,GEIH_O18,GEIH_O19,GEIH_O20 = Dog
GEIH_LIST_O=[GEIH_O08,GEIH_O09,GEIH_O10,GEIH_O11,GEIH_O12,GEIH_O13,GEIH_O14,
    GEIH_O15,GEIH_O16,GEIH_O17,GEIH_O18,GEIH_O19,GEIH_O20]

In [ ]:
GEIH_LIST_D = change_label(GEIH_LIST_D)
GEIH_LIST_O = change_label(GEIH_LIST_O)

In [ ]:
GEIH_LIST_D[12].columns.values[6]="RAMA4D_D"
GEIH_LIST_O[12].columns.values[6]="RAMA4D"
GEIH_LIST_O[12].columns.values[9]="FEX_C_2011"
GEIH_LIST_D[12].columns.values[9]="FEX_C_2011"

In [ ]:
GEIH_LIST_O=Sort("RAMA4D",GEIH_LIST_O)
GEIH_LIST_D=Sort("RAMA4D_D",GEIH_LIST_D)

In [ ]:
GEIH_G08,GEIH_G09,GEIH_G10,GEIH_G11,GEIH_G12,GEIH_G13,GEIH_G14,GEIH_G15,GEIH_G16,GEIH_G17,GEIH_G18,GEIH_G19,GEIH_G20 = 0,0,0,0,0,0,0,0,0,0,0,0,0 
GEIH_G=[GEIH_G08,GEIH_G09,GEIH_G10,GEIH_G11,GEIH_G12,GEIH_G13,GEIH_G14,
    GEIH_G15,GEIH_G16,GEIH_G17,GEIH_G18,GEIH_G19,GEIH_G20]

In [ ]:
GEIH_GD08,GEIH_GD09,GEIH_GD10,GEIH_GD11,GEIH_GD12,GEIH_GD13,GEIH_GD14,GEIH_GD15,GEIH_GD16,GEIH_GD17,GEIH_GD18,GEIH_GD19,GEIH_GD20 = 0,0,0,0,0,0,0,0,0,0,0,0,0 
GEIH_GD=[GEIH_GD08,GEIH_GD09,GEIH_GD10,GEIH_GD11,GEIH_GD12,GEIH_GD13,GEIH_GD14,
    GEIH_GD15,GEIH_GD16,GEIH_GD17,GEIH_GD18,GEIH_GD19,GEIH_GD20]

In [ ]:
for i in range(len(GEIH_LIST_D)):
    index=GEIH_LIST_D[i]["RAMA4D_D"].drop_duplicates().sort_values()
    GEIH_GD[i]= pd.DataFrame(index=index,columns=["Number_Workers_D","fact_c_D"])

In [ ]:
for i in range(len(GEIH_LIST_O)):
    index=GEIH_LIST_O[i]["RAMA4D"].drop_duplicates().sort_values()
    GEIH_G[i]= pd.DataFrame(index=index,columns=["Number_Workers","fact_c"])

In [ ]:
for j in GEIH_LIST_O:
    condition = j["P6870"].apply(lambda x: x == 9)
    if condition.any():
        GEIH_G=calculate_avg_or_sum("RAMA4D", "RAMA4D", "Number_Workers", GEIH_G, GEIH_LIST_O, "count")
        GEIH_G=calculate_avg_or_sum("RAMA4D", "FEX_C_2011", "fact_c", GEIH_G, GEIH_LIST_O, "sum") 

In [ ]:
for j in GEIH_LIST_D:
    condition = j["P7360"].apply(lambda x: x == 9)
    if condition.any():
        GEIH_GD=calculate_avg_or_sum("RAMA4D_D", "RAMA4D_D", "Number_Workers_D", GEIH_GD, GEIH_LIST_D, "count")
        GEIH_GD=calculate_avg_or_sum("RAMA4D_D", "FEX_C_2011", "fact_c_D", GEIH_GD, GEIH_LIST_D, "sum") 

In [ ]:
for df  in GEIH_GD:
    df["Real_Rama_D"] = df.apply(lambda row: mul_of_rows(row,"Number_Workers_D","fact_c_D"), axis=1)

In [ ]:
for df  in GEIH_G:
    df["Real_Rama"] = df.apply(lambda row: mul_of_rows(row,"Number_Workers","fact_c"), axis=1)

In [ ]:
GEIH08,GEIH09,GEIH10,GEIH11,GEIH12,GEIH13,GEIH14,GEIH15,GEIH16,GEIH17,GEIH18,GEIH19,GEIH20 = 0,0,0,0,0,0,0,0,0,0,0,0,0 
GEIH=[GEIH08,GEIH09,GEIH10,GEIH11,GEIH12,GEIH13,GEIH14,
    GEIH15,GEIH16,GEIH17,GEIH18,GEIH19,GEIH20]

In [ ]:
for i, geih_df in enumerate(GEIH_G):
    data_df = Data_Gruop[i]  # Obtener el DataFrame correspondiente de Data_Gruop
    
    # Reindexar el DataFrame de GEIH_G para que tenga los mismos índices que el de Data_Gruop
    geih_df = geih_df.reindex(index=data_df.index)
    
    # Si deseas eliminar las filas que no coinciden en índices, puedes hacerlo así
    geih_df.dropna(inplace=True)
    
    # Ahora, geih_df contiene solo las filas que coinciden en índices con data_df
    
    # Si deseas guardar el DataFrame actualizado de GEIH_G de nuevo en la lista, puedes hacerlo así
    GEIH_G[i] = geih_df



In [ ]:
for i, geih_df in enumerate(GEIH_GD):
    data_df = Data_Gruop[i]  # Obtener el DataFrame correspondiente de Data_Gruop
    
    # Reindexar el DataFrame de GEIH_G para que tenga los mismos índices que el de Data_Gruop
    geih_df = geih_df.reindex(index=data_df.index)
    
    # Si deseas eliminar las filas que no coinciden en índices, puedes hacerlo así
    geih_df.dropna(inplace=True)
    
    # Ahora, geih_df contiene solo las filas que coinciden en índices con data_df
    
    # Si deseas guardar el DataFrame actualizado de GEIH_G de nuevo en la lista, puedes hacerlo así
    GEIH_GD[i] = geih_df


In [ ]:
for i, geih_df in enumerate(GEIH_G):
    data_df = GEIH_GD[i]  # Obtener el DataFrame correspondiente de Data_Gruop
    
    # Reindexar el DataFrame de GEIH_G para que tenga los mismos índices que el de Data_Gruop
    geih_df = geih_df.reindex(index=data_df.index)
    
    # Si deseas eliminar las filas que no coinciden en índices, puedes hacerlo así
    geih_df.dropna(inplace=True)
    
    # Ahora, geih_df contiene solo las filas que coinciden en índices con data_df
    
    # Si deseas guardar el DataFrame actualizado de GEIH_G de nuevo en la lista, puedes hacerlo así
    GEIH[i] = geih_df


In [ ]:
Tuc = create_index(Data_Gruop)
X=['Ocupados']
Empleo = pd.DataFrame(index=Index, columns=X)
Empleo['Desocupados'] = 'NaN'

In [ ]:
T=[]
Y=[]
for i in GEIH_G:
    X = i['Real_Rama'].sum()
    T.append(X)

for i in GEIH_GD:
    W = i['Real_Rama_D'].sum()
    Y.append(W)


In [ ]:
Empleo['Ocupados'] = T
Empleo['Desocupados'] = Y

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Supongamos que 'df' es tu DataFrame con un índice y valores que deseas graficar
# Asegúrate de que 'df' esté cargado correctamente con tus datos.

# Crear la gráfica
# Tamaño de la gráfica (ajusta según tus preferencias)
plt.plot(df.index, Empleo['Ocupados'], marker='o', linestyle='-', color='b', label='Datos')  # Trazar los datos

# Personalizar la gráfica
plt.xlabel('Año')  # Etiqueta del eje X (ajusta el texto según tus preferencias)
plt.ylabel('Valores')  # Etiqueta del eje Y (ajusta el texto según tus preferencias)
plt.title('Ocupados')  # Título de la gráfica (ajusta el texto según tus preferencias)
plt.grid(True)  # Mostrar una cuadrícula en el gráfico si es necesario
plt.legend()  # Mostrar leyenda si es necesario

# Mostrar la gráfica
plt.show()
